In [ ]:
from google.colab import drive 
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import pandas as pd

In [ ]:
def counting():
  num = 0
  while True:
    yield num
    num = num + 1

def convert_acronyms(txt):
  txt = re.sub('[vV]/[vV]', 'về việc', txt)
  txt = re.sub(' PTTH', 'phát thanh truyền hình', txt)
  txt = re.sub(' TTTT', ' Thông tin và Truyền thông', txt)
  return txt



def clean_list(txt, cnt, add_list_token = True):
  count = 0
  if add_list_token is False:
    for x in cnt:
      count = x
      break
  # clean numbered list type (1. 2. 3. ...)
  if re.match(r"\d+(\.|:).*", txt):
    for x in cnt:
      count = x
      break
    if count == 0:
      txt = re.sub(r'\d+(\.|:)', '[LIST]', txt, count=1)
    else:
      txt = re.sub(r'\d+(\.|:)', '', txt, count=1)

  # clean numbered list type (a) b) c) ...)
  elif re.match(r"\w[)]+", txt):
    for x in cnt:
      count = x
      break
    if count == 0:
      txt = re.sub(r'[a-z][)]', '[LIST]', txt, count=1)
    else:
      txt = re.sub(r'[a-z][)]', '', txt, count=1)

  # clean bulleted list type (- - - ...)
  elif re.match(r"[-]+", txt):
    for x in cnt:
      count = x
      break
    if count == 0:
      txt = re.sub('[-]', '[LIST]', txt, count=1)
    else:
      txt = re.sub('[-]', '', txt, count=1)

  # clean bulleted list type (+ + + ...)
  elif re.match(r"[+]+", txt):
    for x in cnt:
      count = x
      break
    if count == 0:
      txt = re.sub('[+]', '[LIST]', txt, count=1)
    else:
      txt = re.sub('[+]', '', txt, count=1)

  if re.match(r"[.;:!?]", txt[len(txt)-1]):
    pass
  else:
    txt = txt + '.'
  return txt


def clean_data(data, add_list_token = True):
  cnt = counting()
  
  res = re.split('\n', data)
  res = [value for value in res if value != '']
  res = [convert_acronyms(value) for value in res]
  res = [clean_list(value, cnt, add_list_token) for value in res]

  final_res = ''
  for value in res:
    final_res = final_res + ' ' + value 
  return final_res

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data.xlsx - Question - Answer.csv",encoding='utf-8') 
data.head()

,Contributed,Group,Intent,Root Usersays,Answer,Extract Answer,Paragraph,Len,Tags,Checked,Note
0,Cho anh biết ở Trung Quốc có cung cấp giấy phé...,\nHậu kiểm,Giấy phép cung cấp dịch vụ OTT TV qua mạng int...,Trung Quốc có cấp giấy phép cung cấp dịch vụ O...,"Không, Trung Quốc không cho phép các doanh ngh...",NaN,NaN,100,Không sử dụng,True,NaN
1,Thế nào là multiscreen trong truyền hình em nh...,"\nĐánh giá về một số xu hướng nghe, xem",Thuật ngữ multiscreen,Thuật ngữ multiscreen trong truyền hình được h...,Xem truyền hình với nhiều thiết bị hiển thị kh...,NaN,NaN,101,NaN,False,NaN
2,Những người nào được gọi là thanh tra viên? Có...,Phòng Thanh tra pháp chế (140 câu)\n,Người được gọi là Thanh tra viên,Những người làm công tác thanh tra tại các Cục...,Được quy định tại điều 34 Luật Thanh tra “Ngườ...,Người được giao thực hiện nhiệm vụ thanh tra c...,Người được giao thực hiện nhiệm vụ thanh tra c...,101,Sử dụng,True,56/2010/QH12
3,"Tôi thắc mắc là báo chí nước ngoài, cơ quan đạ...",CUCBC_BO CAU HOI DAP LIEN QUAN QLNN VE BAO CHI,Báo chí nước ngoài thực hiện hoạt động báo chí...,"Báo chí nước ngoài, cơ quan đại diện nước ngoà...",Phải được sự cho phép của các cơ quan có thẩm ...,NaN,NaN,102,Không sử dụng,True,NaN
4,Hiện nay có những loại mạng máy tính nào ?\nMạ...,Tổng Hợp Câu Hỏi Aita V2,Các loại mạng máy tính,Có các loại mạng máy tính nào?,Theo Wikipedia: Hệ thống mạng máy tính bao gồm...,NaN,NaN,102,NaN,False,NaN


In [ ]:
data = data[data['Tags'].notna()]
data.drop(data.loc[data['Tags']=='Không sử dụng'].index, inplace=True)

In [ ]:
data['Extract Answer ft 1']=[clean_data(str(value)) for value in data['Extract Answer']]
data['Extract Answer ft 2']=[clean_data(str(value), add_list_token = False) for value in data['Extract Answer']]
data['Paragraph ft 1']=[clean_data(str(value)) for value in data['Paragraph']]
data['Paragraph ft 2']=[clean_data(str(value), add_list_token = False) for value in data['Paragraph']]

In [ ]:
data.head()

,Contributed,Group,Intent,Root Usersays,Answer,Extract Answer,Paragraph,Len,Tags,Checked,Note,Extract Answer ft 1,Extract Answer ft 2,Paragraph ft 1,Paragraph ft 2
2,Những người nào được gọi là thanh tra viên? Có...,Phòng Thanh tra pháp chế (140 câu)\n,Người được gọi là Thanh tra viên,Những người làm công tác thanh tra tại các Cục...,Được quy định tại điều 34 Luật Thanh tra “Ngườ...,Người được giao thực hiện nhiệm vụ thanh tra c...,Người được giao thực hiện nhiệm vụ thanh tra c...,101,Sử dụng,True,56/2010/QH12,Người được giao thực hiện nhiệm vụ thanh tra ...,Người được giao thực hiện nhiệm vụ thanh tra ...,Người được giao thực hiện nhiệm vụ thanh tra ...,Người được giao thực hiện nhiệm vụ thanh tra ...
5,như thế nào là công ty tư vấn dự án vậy?\nDự á...,Tổng Hợp Câu Hỏi Aita V2,Khái niệm dự án đầu tư công,Dự án đầu tư công là gì?,Theo quy định tại Điều 4 Luật đầu tư công số 3...,là dự án sử dụng toàn bộ hoặc một phần vốn đầu...,11. Cơ quan chuyên môn quản lý đầu tư công là ...,103,Sử dụng,True,39/2019/QH14,là dự án sử dụng toàn bộ hoặc một phần vốn đầ...,là dự án sử dụng toàn bộ hoặc một phần vốn đầ...,[LIST] Cơ quan chuyên môn quản lý đầu tư công...,Cơ quan chuyên môn quản lý đầu tư công là đơ...
8,Bạn hãy giải thích cho tôi phân cấp quản lý nh...,Tổng Hợp Câu Hỏi Aita V2,Phân cấp quản lý nhà nước về đầu tư công,Phân cấp quản lý nhà nước về đầu tư công là gì?,Theo quy định tại Điều 4 Luật đầu tư công số 3...,là dự án sử dụng toàn bộ hoặc một phần vốn đầu...,20. Nợ đọng xây dựng cơ bản là giá trị khối lư...,103,Sử dụng,True,39/2019/QH14,là dự án sử dụng toàn bộ hoặc một phần vốn đầ...,là dự án sử dụng toàn bộ hoặc một phần vốn đầ...,[LIST] Nợ đọng xây dựng cơ bản là giá trị khố...,Nợ đọng xây dựng cơ bản là giá trị khối lượn...
9,Như thế nào là địa chỉ cổng dữ liệu quốc gia,Tổng Hợp Câu Hỏi Aita V2,Địa chỉ cổng dữ liệu quốc gia,Địa chỉ Cổng dữ liệu quốc gia là gì,"Tại Khoản 10, Điều 3 của Nghị định số 47 năm 2...",Cổng dữ liệu quốc gia có địa chỉ data.gov.vn,Cổng dữ liệu quốc gia: là điểm đầu mối truy cậ...,103,Sử dụng,True,NaN,Cổng dữ liệu quốc gia có địa chỉ data.gov.vn.,Cổng dữ liệu quốc gia có địa chỉ data.gov.vn.,Cổng dữ liệu quốc gia: là điểm đầu mối truy c...,Cổng dữ liệu quốc gia: là điểm đầu mối truy c...
11,Định nghĩa về chương trình Truyền hình ?\nChươ...,Phòng Thanh tra pháp chế (140 câu)\n,Khái niệm Chương trình truyền hình,Chương trình Truyền hình là gì?,"Là tập hợp các tin, bài trên báo hình theo một...","Là tập hợp các tin, bài trên báo hình theo một...",b) “Sản phẩm báo chí” trong báo hình là kênh t...,104,Sử dụng,True,03/2018/TT-BTTTT,"Là tập hợp các tin, bài trên báo hình theo mộ...","Là tập hợp các tin, bài trên báo hình theo mộ...",[LIST] “Sản phẩm báo chí” trong báo hình là k...,“Sản phẩm báo chí” trong báo hình là kênh tr...


In [ ]:
data.to_csv("/content/drive/MyDrive/Colab Notebooks/Data.csv")